In [321]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA

import statsmodels.api as sm

In [322]:
import warnings
warnings.filterwarnings('ignore')

In [323]:
df = pd.read_csv('zillow_data.csv')
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [324]:
df.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04'],
      dtype='object', length=272)

In [325]:
drop = list(df.columns[[0,2,3,4,5,6]])
df = df.drop(drop, axis = 1)
df.head()

,RegionName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,60657,334200.0,335400.0,336500.0,337600.0,338500.0,339500.0,340400.0,341300.0,342600.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,75070,235700.0,236900.0,236700.0,235400.0,233300.0,230600.0,227300.0,223400.0,219600.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,77494,210400.0,212200.0,212200.0,210700.0,208300.0,205500.0,202500.0,199800.0,198300.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,60614,498100.0,500900.0,503100.0,504600.0,505500.0,505700.0,505300.0,504200.0,503600.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,79936,77300.0,77300.0,77300.0,77300.0,77400.0,77500.0,77600.0,77700.0,77700.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [326]:
df.RegionName.nunique()

14723

15K possible zip codes, and you want me to pick the best ones? Seriously yall need Jesus.

# Helper Function Space

### Transform row

In [7]:
# Create a datetime series from a row in our dataframe
def transform_row(row, df):
    TS = df.iloc[row]
    RegionName = TS.iloc[0]
    TS.drop('RegionName', inplace = True, axis = 0)
    TS.index = pd.to_datetime(TS.index)
    TS.dropna(inplace = True)
    return TS, RegionName

### Stationarity evaluation, return most stationary series

In [8]:
def best_transform(df):
    #This function will evaluate a given pandas timeseries Dataframe (n x 1) across multiple transforms for stationarity,
    #and then return the most stationary TS, along with the descriptors p for Dickey-Fuller test,
    #a string description of the type of the best transformation, and
    #a stat describing the nth order of the transform where applicable.
    
    ###
    #Create Pandas core series from (n x 1) dataframe
    TS = df[df.columns[0]]
    
    ###
    #Evaluate TS without transformation
    #Store Dickey-Fuller(DF) p value
    dftest = adfuller(TS.values)
    p_as_is = dftest[1]
    ###
    
    ###
    #Stationarity of ln transform
    #Store Dickey-Fuller(DF) p value
    dftest = adfuller(np.log(TS.values))
    p_log = dftest[1]
    ###
    
    ###
    #Stationarity of curve w/ rolling mean subtracted
    #Checks rolling mean across iterations of 2 - 6 periods
    #Selects best DF p value
    roll_result = {}
    roll_param = {}
    p_rollmean = []

    for n in range(2,7):
        rollmean = TS.rolling(window = n).mean()
        minus_rollmean = TS - rollmean
        minus_rollmean.dropna(inplace = True)
        dftest = adfuller(minus_rollmean)
        p_rollmean.append(dftest[1])
        roll_result[dftest[1]] = rollmean
        roll_param[dftest[1]] = n

    p_rollmean = min(p_rollmean)
    ###
    
    ###
    #Stationarity of curve w/ ewm
    #Checks exponentially weighted mean across iterations of 1-6 periods
    #Selects best DF p value
    decay_result = {}
    decay_param = {}
    p_ewm = []

    for n in range(1,7):
        decay_rollmean = TS.ewm(halflife = n).mean()
        minus_ewm = TS - decay_rollmean
        minus_ewm.dropna(inplace = True)
        dftest = adfuller(minus_ewm)
        p_ewm.append(dftest[1])
        decay_result[dftest[1]] = decay_rollmean
        decay_param[dftest[1]] = n
    
    p_ewm = min(p_ewm)
    ###
    
    ###
    #Stationarity check of differenced dataframe
    #Checks across [1,2,3,4,5,6,7,12,30,365] periods
    diff_result = {}
    diff_param = {}
    p_diff = []
    
    for n in [1,2,3,4,5,6,7,12,30,365]:
        try:
            diff_df = df.diff(periods = n)
            diff_TS = diff_df[diff_df.columns[0]].dropna().values
            dftest = adfuller(diff_TS)
            p_diff.append(dftest[1])
            diff_result[dftest[1]] = diff_TS
            diff_param[dftest[1]] = n
        except:
            continue

    p_diff = min(p_diff)
    
    ###
    #Select lowest P-Value, most staionary curve
    p = min(p_as_is, p_log, p_rollmean, p_ewm, p_diff)
    
    #Return most stationary TS
    if p == p_as_is:
        transform = 'None'
        stat = None
        return TS, p, transform, stat
    
    if p == p_log:
        transform = 'Log'
        stat = None
        return np.log(TS), p, transform, stat
    
    if p == p_rollmean:
        transform = 'Rolling Mean'
        stat = roll_param[p]
        return roll_result[p], p, transform, stat
    
    if p == p_ewm:
        transform = 'EWM'
        stat = decay_param[p]
        return decay_result[p], p, transform, stat
    
    if p == p_diff:
        transform = 'Difference'
        stat = diff_param[p]
        return diff_result[p], p, transform, stat

### Iterative ARIMA

In [ ]:
def Iterative_ARIMA(series):
    #This function takes in a timeseries series, iteratively conducts ARIMA models of select PDQ values,
    #selects the best model by minimizing AIC, then returns AIC score, optimized PDQ, and model RMSE.
    
    #####
    #Function iteratively models select permutations of PDQ to generate a list of AIC values.
    #Dictionaries tie model paramaters (PDQ) and measure of error (RMSE) to AIC results
    #by using AIC as a key value. The function will then select the lowest AIC value from the list,
    #then lookup the associated paramters and error in the dictionaries.
    p = range(0,4)
    d = range(0,3)
    q = range(0,4)
    
    PDQ_result_lookup_by_AIC = {}
    RMSE_result_lookup_by_AIC = {}
    AIC = []
    
    for j in d:
        for i in p:
            for i in q:
                try:
                    ARIMA_model = ARIMA(endog = series.values, dates = series.index, order = (p[i],d[j],q[i]))
                    fit = ARIMA_model.fit()
                    AIC.append(fit.aic)
                    PDQ_result_lookup_by_AIC[fit.aic] = (p[i],d[j],q[i])
                    RMSE_result_lookup_by_AIC[fit.aic] = np.sqrt(np.mean(fit.resid**2))
                except:
                    continue
    best = min(AIC)
    pdq = PDQ_result_lookup_by_AIC[best]
    rmse = RMSE_result_lookup_by_AIC[best]
    return best, pdq, rmse

# Iterative ARIMA w/ AIC penalties for high P + Q

In [ ]:
def Penalty_Iterative_ARIMA(series):
    #This function takes in a timeseries series, iteratively conducts ARIMA models of select PDQ values,
    #selects the best model by minimizing AIC, then returns AIC score, optimized PDQ, and model RMSE.
    
    #####
    #Function iteratively models select permutations of PDQ to generate a list of AIC values.
    #Dictionaries tie model paramaters (PDQ) and measure of error (RMSE) to AIC results
    #by using AIC as a key value. The function will then select the lowest AIC value from the list,
    #then lookup the associated paramters and error in the dictionaries.
    p = range(0,4)
    d = range(0,3)
    q = range(0,4)
    
    PDQ_result_lookup_by_AIC = {}
    RMSE_result_lookup_by_AIC = {}
    AIC = []
    
    for j in d:
        for i in p:
            for i in q:
                try:
                    ARIMA_model = ARIMA(endog = series.values, dates = series.index, order = (p[i],d[j],q[i]))
                    fit = ARIMA_model.fit()
                    if p[i] + q[i] > 3:
                        aic = (fit.aic) * ((p[i] + q[i])/3)
                    else:
                        aic = fit.aic
                    AIC.append(aic)
                    PDQ_result_lookup_by_AIC[aic] = (p[i],d[j],q[i])
                    RMSE_result_lookup_by_AIC[aic] = np.sqrt(np.mean(fit.resid**2))
                except:
                    continue
    best = min(AIC)
    pdq = PDQ_result_lookup_by_AIC[best]
    rmse = RMSE_result_lookup_by_AIC[best]
    return best, pdq, rmse

### ARIMA Regressive Prediction and Forecasting

In [245]:
def ARIMA_adcanced_predict(series, pdq, periods):
    #This function converts original series values to a list ('History'). A model is run using the specified
    #PDQ parameters, and then iteratively forecasts only one additional period at a time,
    #regressively adding in the forecast value. It does this for a specified n periods in range.
    #Returns the final forecasted result at the end of the specified period.
    history = list(series.values)
    for n in range(1,periods):
        try:
            ARIMA_model = ARIMA(endog = history, order = pdq)
            fit = ARIMA_model.fit()
            history.append(fit.forecast(1)[0])
        except:
            ARIMA_model = ARIMA(endog = history, order = [0,0,0])
            fit = ARIMA_model.fit()
            history.append(fit.forecast(1)[0])
    forecast_result = history[-1]
    return forecast_result

### ARIMA functional forecasting

In [296]:
def ARIMA_basic_forecast(series, pdq, periods):
    ARIMA_model = ARIMA(endog = series.values, dates = series.index, order = pdq)
    fit = ARIMA_model.fit()
    forecast = fit.forecast(periods)
    forecast_result = forecast[0][-1]
    forecast_lower_bound = forecast[2][-1][0]
    forecast_upper_bound = forecast[2][-1][1]
    return forecast_result, forecast_lower_bound, forecast_upper_bound

# MAKING THE BIGGEST BOY

In [256]:
Models = None
Models = pd.DataFrame(columns = ['Region Name', 'AIC', 'PDQ', 'RMSE', 'ROI'])

### Testing w/ subset

In [233]:
import timeit
start = timeit.default_timer()

for i in range(0, 10):
    print(i)
    TS, RegionName = transform_row(i,df)
    aic, pdq, rmse = Iterative_ARIMA(TS)
    forecast_result = ARIMA_predict(TS, pdq, 60)

    Models = Models.append({'Region Name': RegionName,
                            'AIC': aic,
                            'PDQ': pdq,
                            'RMSE': rmse,
                            'ROI': int(forecast_result - TS.values[-1])}, ignore_index = True)

end = timeit.default_timer()
print(end - start)

0
1
2
3
4
5
6
7
8
9
324.8727994000001


In [234]:
Models

,Region Name,AIC,PDQ,RMSE,ROI
0,60657.0,4527.565542,"(3, 2, 3)",1274.610917,20907
1,75070.0,3944.553454,"(3, 2, 3)",428.483796,98923
2,77494.0,4170.705016,"(3, 2, 3)",655.555310,34975
3,60614.0,4604.868165,"(3, 2, 3)",1477.583385,-36804
4,79936.0,3510.977138,"(2, 2, 2)",186.962339,27555
5,77084.0,3509.146553,"(3, 2, 3)",185.068027,39877
6,10467.0,4468.081219,"(3, 2, 3)",1138.848197,137032
7,60640.0,4618.753297,"(3, 2, 3)",1510.424320,27387
8,77449.0,3576.995769,"(3, 2, 3)",210.097993,-59693
9,94109.0,5555.498034,"(2, 2, 2)",9119.589404,1108381


In [243]:
print(f'n days to model and predict all counties this way: {((len(df.index)*(325/10))/(60**2))/24}')

n days to model and predict all counties this way: 5.538165509259259


### Testing w/ subset

In [257]:
start = timeit.default_timer()

for i in range(0, 25):
    print(i)
    TS, RegionName = transform_row(i,df)
    aic, pdq, rmse = Iterative_ARIMA(TS)
    forecast_result, lower, upper = ARIMA_basic_forecast(TS, pdq, 60)

    Models = Models.append({'Region Name': RegionName,
                            'AIC': aic,
                            'PDQ': pdq,
                            'RMSE': rmse,
                            'ROI': int(forecast_result - TS.values[-1])}, ignore_index = True)

end = timeit.default_timer()
print(end - start)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
218.39361689999714


In [258]:
print(f'n days to model and predict all counties this way: {((len(df.index)*(218/25))/(60**2))/24}')

n days to model and predict all counties this way: 1.4859324074074076


In [259]:
Models

,Region Name,AIC,PDQ,RMSE,ROI
0,60657.0,4527.565542,"(3, 2, 3)",1274.610917,20725
1,75070.0,3944.553454,"(3, 2, 3)",428.483796,102249
2,77494.0,4170.705016,"(3, 2, 3)",655.555310,37196
3,60614.0,4604.868165,"(3, 2, 3)",1477.583385,-37592
4,79936.0,3510.977138,"(2, 2, 2)",186.962339,28072
5,77084.0,3509.146553,"(3, 2, 3)",185.068027,40600
6,10467.0,4468.081219,"(3, 2, 3)",1138.848197,139615
7,60640.0,4618.753297,"(3, 2, 3)",1510.424320,31022
8,77449.0,3576.995769,"(3, 2, 3)",210.097993,30775
9,94109.0,5555.498034,"(2, 2, 2)",9119.589404,1127439


# Smaller biggest boy

In [307]:
Models = None
Models = pd.DataFrame(columns = ['Region Name', 'AIC', 'PDQ', 'RMSE', 'ROI Proj', 'Worst ROI', 'Best ROI'])

In [308]:
start = timeit.default_timer()

for i in range(0, 25):
    print(i)
    TS, RegionName = transform_row(i,df)
    aic, pdq, rmse = Iterative_ARIMA(TS)
    forecast_result, lower, upper = ARIMA_basic_forecast(TS, pdq, 60)

    Models = Models.append({'Region Name': RegionName,
                            'AIC': aic,
                            'PDQ': pdq,
                            'RMSE': rmse,
                            'ROI Proj': int(forecast_result - TS.values[-1]),
                            'Worst ROI': int(lower - TS.values[-1]),
                            'Best ROI': int(upper - TS.values[-1])}, ignore_index = True)

end = timeit.default_timer()
print(end - start)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
217.75409720000243


In [309]:
Models

,Region Name,AIC,PDQ,RMSE,ROI Proj,Worst ROI,Best ROI
0,60657.0,4527.565542,"(3, 2, 3)",1274.610917,20725,-446438,487890
1,75070.0,3944.553454,"(3, 2, 3)",428.483796,102249,52616,151883
2,77494.0,4170.705016,"(3, 2, 3)",655.555310,37196,-100010,174402
3,60614.0,4604.868165,"(3, 2, 3)",1477.583385,-37592,-720087,644901
4,79936.0,3510.977138,"(2, 2, 2)",186.962339,28072,-86593,142737
5,77084.0,3509.146553,"(3, 2, 3)",185.068027,40600,-16649,97849
6,10467.0,4468.081219,"(3, 2, 3)",1138.848197,139615,-158435,437667
7,60640.0,4618.753297,"(3, 2, 3)",1510.424320,31022,-453932,515977
8,77449.0,3576.995769,"(3, 2, 3)",210.097993,30775,-754,62304
9,94109.0,5555.498034,"(2, 2, 2)",9119.589404,1127439,-3226941,5481821


# Lock and Load

In [332]:
Models = None
Models = pd.DataFrame(columns = ['Region Name', 'AIC', 'PDQ', 'RMSE', 'ROI Proj', 'Worst ROI', 'Best ROI'])

In [333]:
start = timeit.default_timer()

for i in range(0, len(df)):
    TS, RegionName = transform_row(i,df)
    aic, pdq, rmse = Iterative_ARIMA(TS)
    forecast_result, lower, upper = ARIMA_basic_forecast(TS, pdq, 60)

    Models = Models.append({'Region Name': RegionName,
                            'AIC': aic,
                            'PDQ': pdq,
                            'RMSE': rmse,
                            'ROI Proj': int(forecast_result - TS.values[-1]),
                            'Worst ROI': int(lower - TS.values[-1]),
                            'Best ROI': int(upper - TS.values[-1])}, ignore_index = True)
    
    end = timeit.default_timer()
    time = end-start
    if time > 60**2:
        Models.to_csv(r"Models_savepoint")
        start = timeit.default_timer()
        print(f'Autosaved at index #{i}. {(i/len(df))*100}% completed.')

Autosaved at index #383. 2.6013720029885214% completed.
Autosaved at index #766. 5.202744005977043% completed.
Autosaved at index #1157. 7.8584527609862125% completed.
Autosaved at index #1532. 10.405488011954086% completed.
Autosaved at index #1913. 12.993275826937445% completed.
Autosaved at index #2294. 15.581063641920803% completed.
Autosaved at index #2672. 18.14847517489642% completed.
Autosaved at index #3047. 20.695510425864295% completed.
Autosaved at index #3424. 23.25612986483733% completed.
Autosaved at index #3802. 25.823541397812942% completed.
Autosaved at index #4166. 28.295863614752427% completed.
Autosaved at index #4543. 30.856483053725466% completed.
Autosaved at index #4916. 33.389934116688174% completed.
Autosaved at index #5278. 35.848672145622494% completed.
Autosaved at index #5649. 38.368539020580045% completed.
Autosaved at index #6024. 40.91557427154792% completed.
Autosaved at index #6406. 43.51015418053386% completed.
Autosaved at index #6774. 46.009644773

In [335]:
Models.shape

(14723, 7)

In [336]:
Models.to_csv(r"Models.csv")